In [1]:
import pandas as pd
import awswrangler as wr
import boto3
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [4]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key, 
                                region_name='us-east-1')

In [9]:
bronze_base = 'raw-anakin'
silver_base = 'silver-darthvader'

In [7]:
wr.catalog.create_database(silver_base, exist_ok=True, boto3_session=session, )

In [12]:
query = """
SELECT 
    name,
    TRY_CAST(rotation_period AS INT) AS rotation_period,
    TRY_CAST(orbital_period AS INT) AS orbital_period,
    TRY_CAST(diameter AS INT) AS diameter,
    IF(climate LIKE '%hot%', TRUE, FALSE) AS is_hot,
    IF(climate LIKE '%arid%', TRUE, FALSE) AS is_arid,
    IF(climate LIKE '%rocky%', TRUE, FALSE) AS is_rocky,
    IF(climate LIKE '%windy%', TRUE, FALSE) AS is_windy,
    IF(climate LIKE '%temperate%', TRUE, FALSE) AS is_temperate,
    IF(climate LIKE '%frigid%', TRUE, FALSE) AS is_frigid,
    IF(climate LIKE '%frozen%', TRUE, FALSE) AS is_frozen,
    IF(climate LIKE '%polluted%', TRUE, FALSE) AS is_polluted,
    IF(climate LIKE '%tropical%', TRUE, FALSE) AS is_tropical,
    CASE 
        WHEN gravity = 'unknown' THEN NULL
        WHEN gravity = 'N/A' THEN NULL
        ELSE CAST(SPLIT(gravity, ' ')[1] AS double)
    END AS gravity,
    TRY_CAST(surface_water AS INT) AS surface_water,
    TRY_CAST(population AS INT) AS population,
    CAST(FROM_ISO8601_TIMESTAMP_NANOS(created) AS TIMESTAMP) AS ts_created,
    CAST(FROM_ISO8601_TIMESTAMP_NANOS(edited) AS TIMESTAMP) AS ts_edited
FROM planets
WHERE name != 'unknown'
"""

In [13]:
df = wr.athena.read_sql_query(sql=query, database=bronze_base, boto3_session=session)

In [17]:
path = 's3://silver-darthvader/planets'

In [18]:
wr.s3.to_parquet(
    df=df,
    path=path,
    dataset=True,
    mode="overwrite",
    database=silver_base,
    table='planets',
    boto3_session=session
)

{'paths': ['s3://silver-darthvader/planets/887fc3928aa54d1fa0df30a703f5cff1.snappy.parquet'],
 'partitions_values': {}}